# Example notebook of a Sample QC analysis

The present notebook serves as a guide of how use the `IDEAL-QC` library to perform a sample quality control. We intend to show a possible use, because each user can adapt it to its particular needs.

The first step is to import the required libraries.

In [1]:
import sys
import os

import pandas as pd

# add parent directory to path
library_path = os.path.abspath('..')
if library_path not in sys.path:
    sys.path.append(library_path)

from cge_comrare_pipeline.SampleQC import SampleQC

In the next widgets the user must input the the paths and filenames needed to perform the sample QC.

1. `input_path`: the path to the folder where the raw data in PLINK (`.bed`, `.bim`, `.fam`) files is.
2. `input_name`: the prefix of the PLINK files. It is not required the name of the three files, since the extension is deduced from the PLINK format.
3. `dependables_path`: path to the folder where external files needed to perform the QC are. In this cases, the QC requires a file with high LD regions. The file should be named as `high-LD-regions.txt`.
4. `output_path`: the path to the folder where the outputs from each steps are going to be.
5. `output_name`: the prefix of the cleaned PLINK files.

In [7]:
import ipywidgets as widgets
from IPython.display import display

# Create interactive widgets for input
input_path = widgets.Text(
    value='/mnt/0A2AAC152AABFBB7/data/rawdata-sexupdated/inputData',
    description='Path to input plink1.9 files:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='50%')
)

input_name = widgets.Text(
    value='luxgiant_data_combined_12098-updated-sex',
    description='Name of the plink1.9 files:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='50%')
)

dependables_path = widgets.Text(
    value='/mnt/0A2AAC152AABFBB7/data/rawdata-sexupdated/dependables/',
    description='Path to dependable files:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='50%')
)

output_path = widgets.Text(
    value='/mnt/0A2AAC152AABFBB7/data/rawdata-sexupdated/outputData/',
    description='Path to output files:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='50%')
)
output_name = widgets.Text(
    value='luxgiant_res',
    description='Name of the resulting files:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='50%')
)
# Display the widgets
display(input_path, input_name, dependables_path, output_path, output_name)

# Function to get the text parameter values
def get_params():
    return input_path.value, input_name.value, dependables_path.value, output_path.value, output_name.value

Text(value='/mnt/0A2AAC152AABFBB7/data/rawdata-sexupdated/inputData', description='Path to input plink1.9 file…

Text(value='luxgiant_data_combined_12098-updated-sex', description='Name of the plink1.9 files:', layout=Layou…

Text(value='/mnt/0A2AAC152AABFBB7/data/rawdata-sexupdated/dependables/', description='Path to dependable files…

Text(value='/mnt/0A2AAC152AABFBB7/data/rawdata-sexupdated/outputData/', description='Path to output files:', l…

Text(value='luxgiant_res', description='Name of the resulting files:', layout=Layout(width='50%'), style=TextS…

In [8]:
# Use the parameter values
path_params = get_params()
print(f"Input Path: {path_params[0]}")
print(f"Input Name: {path_params[1]}")
print(f"Dependables: {path_params[2]}")
print(f"Output Path: {path_params[3]}")
print(f"Output Name: {path_params[4]}")

Input Path: /mnt/0A2AAC152AABFBB7/data/rawdata-sexupdated/inputData
Input Name: luxgiant_data_combined_12098-updated-sex
Dependables: /mnt/0A2AAC152AABFBB7/data/rawdata-sexupdated/dependables/
Output Path: /mnt/0A2AAC152AABFBB7/data/rawdata-sexupdated/outputData/
Output Name: luxgiant_res


In the next widgets, please provide the parameters to run the required commands. Most of them are common `PLINK1.9` and `PLINK2.0` parameters. 

In [9]:
# Create interactive widgets for list input
ind_par = widgets.Textarea(
    value='50, 5, 0.2',
    description='indep pairwise (comma-separated):',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='50%')
)

mind = widgets.FloatText(
    value=0.2,  # Default value
    description='mind (float):',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='50%')
)

sex_check = widgets.Textarea(
    value = '',
    description='sex check (comma-separated):',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='50%')
)

het_deviation = widgets.FloatText(
    value=3,  # Default value
    description='deviation from the mean heterozigosity rate (float):',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='50%')
)

maf = widgets.FloatText(
    value=0.01,  # Default value
    description='maf (float):',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='50%')
)

kingship = widgets.FloatText(
    value=0.354,  # Default value
    description='mind (float):',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='50%')
)

# display the widgets
display(ind_par, mind, sex_check,het_deviation, maf, kingship)

def get_sample_qc_params():

    sample_qc_params = dict()

    indep = ind_par.value.split(',')
    sex = sex_check.value.split(',')

    sample_qc_params['maf'] = maf.value
    sample_qc_params['mind']= mind.value
    sample_qc_params['kingship'] = kingship.value
    sample_qc_params['indep'] = [int(indep[0]), int(indep[1]), float(indep[2])]
    sample_qc_params['het_deviation'] = het_deviation.value
    if sex!=['']:
        sample_qc_params['sex_check'] = [float(x) for x in sex]
    else:
        sample_qc_params['sex_check'] = []

    return sample_qc_params

Textarea(value='50, 5, 0.2', description='indep pairwise (comma-separated):', layout=Layout(width='50%'), styl…

FloatText(value=0.2, description='mind (float):', layout=Layout(width='50%'), style=DescriptionStyle(descripti…

Textarea(value='', description='sex check (comma-separated):', layout=Layout(width='50%'), style=TextStyle(des…

FloatText(value=3.0, description='deviation from the mean heterozigosity rate (float):', layout=Layout(width='…

FloatText(value=0.01, description='maf (float):', layout=Layout(width='50%'), style=DescriptionStyle(descripti…

FloatText(value=0.354, description='mind (float):', layout=Layout(width='50%'), style=DescriptionStyle(descrip…

In [10]:
sample_params = get_sample_qc_params()
sample_params

{'maf': 0.01,
 'mind': 0.2,
 'kingship': 0.354,
 'indep': [50, 5, 0.2],
 'het_deviation': 3.0,
 'sex_check': []}

Initialize the class `SampleQC`.

In [11]:
sample = SampleQC(
    input_path      =input_path.value,
    input_name      =input_name.value,
    output_path     =output_path.value,
    output_name     =output_name.value,
    dependables_path=dependables_path.value,
)

FileNotFoundError: input_path or output_path is not a valid path

Execute the pipeline steps of the sample quality control. Since the ides of a notebook is to build a more interactive interface, the next steps do not drop the samples failing QC, they just find the samples.

In [ ]:
sample_qc_steps = {
    'rename SNPs'           : (sample.execute_rename_snps, (True,)),
    'hh_to_missing'         : (sample.execute_haploid_to_missing, ()),
    'ld_pruning'            : (sample.execute_ld_pruning, (sample_params['indep'],)),
    'miss_genotype'         : (sample.execute_miss_genotype, (sample_params['mind'],)),
    'sex_check'             : (sample.execute_sex_check, (sample_params['sex_check'])),
    'heterozygosity'        : (sample.execute_heterozygosity_rate, (sample_params['maf'],)),
    'duplicates_relatedness': (sample.execute_duplicate_relatedness, (sample_params['kingship'], True,))
}

step_description = {
    'rename SNPs'           : 'Rename SNPs to chr:pos:ref:alt',
    'hh_to_missing'         : 'Solve hh warnings by setting to missing',
    'ld_pruning'            : 'Perform LD pruning',
    'miss_genotype'         : 'Get samples with high missing rate',
    'sex_check'             : 'Get samples with discordant sex information',
    'heterozygosity'        : 'Get samples with high heterozygosity rate',
    'duplicates_relatedness': 'Get samples with high relatedness rate or duplicates'
}

for name, (func, params) in sample_qc_steps.items():
    print(f"\033[1m{step_description[name]}.\033[0m")
    func(*params)

Here a small dashboard with a report of the call rate missingness is shown. The cap on the Y-axis can be selected without re-running the whole pipeline, so it can be selected according to each user need. Moreover, the plots could help to choose the best call rate threshold according to the data.

In [ ]:
fail_call_rate = sample.report_call_rate(
            directory    =sample.results_dir, 
            filename     =sample.call_rate_miss, 
            threshold    =sample_params['mind'],
            plots_dir    =sample.plots_dir, 
            y_axis_cap   =10
        )

Now, the samples failing sex check are collected and a plot is shown where the user can check the number of problematic samples.

In [ ]:
fail_sexcheck = sample.report_sex_check(
            directory          =sample.results_dir, 
            sex_check_filename =sample.sexcheck_miss, 
            xchr_imiss_filename=sample.xchr_miss,
            plots_dir          =sample.plots_dir
        )

Here a small dashboard with a report of the heterozigosity is shown. The cap on the Y-axis can be selected without re-running the whole pipeline, so it can be selected according to each user need. Moreover, the plots could help to choose a different deviation from the mean of the heterozigosity rate. Notice that the analysis has been divided for SNPs having a MAF of less than 1% and those above that threshold.

In [ ]:
fail_het_greater = sample.report_heterozygosity_rate(
            directory           = sample.results_dir, 
            summary_ped_filename= sample.summary_greater, 
            autosomal_filename  = sample.maf_greater_miss, 
            std_deviation_het   = sample_params['het_deviation'],
            maf                 = sample_params['maf'],
            split               = '>',
            plots_dir           = sample.plots_dir
        )

In [ ]:
fail_het_gless= sample.report_heterozygosity_rate(
            directory           = sample.results_dir, 
            summary_ped_filename= sample.summary_less, 
            autosomal_filename  = sample.maf_less_miss, 
            std_deviation_het   = sample_params['het_deviation'],
            maf                 = sample_params['maf'],
            split               = '<',
            plots_dir           = sample.plots_dir
        )

In [12]:
fail_dup_relatednes = pd.read_csv(
            sample.kinship_miss,
            sep=r'\s+',
            engine='python'
        )

# filter samples that failed duplicates and relatedness check
fail_dup_relatednes.columns = ['FID', 'IID']
fail_dup_relatednes['Failure'] = 'Duplicates and relatedness'

All failing samples are collected and concatenated in a single pandas DataFrame and saved. A summary of the amount of samples failing each step is shown. 

In [13]:
df_fails = pd.concat(
    [fail_call_rate, fail_sexcheck, fail_het_greater, fail_het_gless, fail_dup_relatednes],
    ignore_index=True
)

total_fails = df_fails.shape[0]
duplicates = df_fails.duplicated(subset=['FID', 'IID']).sum()
summary = df_fails['Failure'].value_counts().reset_index()

df_fails = df_fails.drop_duplicates(subset=['FID', 'IID'])

df_fails.to_csv(
    os.path.join(sample.fails_dir, 'fail_samples.txt'), sep='\t',
    index=False
)

In [ ]:
summary

In [ ]:
print('Total samples failing QC: ', total_fails)
print('Number of duplicated samples: ', duplicates)
print('Unique samples failing QC: ', total_fails-duplicates)

Finally, the failing samples are dropped and cleaned `PLINK` files are generated.

In [ ]:
sample.execute_drop_samples()
sample.execute_recover_snp_names(rename=True)